In [4]:
class UserManager:
    def __init__(self):
        self.users_file = "users.json"
        self.receivers_file = "receivers.json"

    def load_list(self, file_path):
        if not os.path.exists(file_path):
            return []
        try:
            with open(file_path, "r") as f:
                return json.load(f)
        except json.JSONDecodeError:
            return []

    def save_list(self, file_path, data_list):
        with open(file_path, "w") as f:
            json.dump(data_list, f, indent=4)

    def modify_users(self, choice, name):
        users = self.load_list(self.users_file)
        if choice == '1':  # Add user
            if name not in users:
                users.append(name)
                self.save_list(self.users_file, users)
                messagebox.showinfo("User Added", f"User '{name}' added.")
            else:
                messagebox.showinfo("Info", f"User '{name}' already exists.")
        else:  # Remove user
            if name in users:
                users.remove(name)
                self.save_list(self.users_file, users)
                messagebox.showinfo("User Removed", f"User '{name}' removed.")
            else:
                messagebox.showwarning("Not Found", f"User '{name}' does not exist.")

    def modify_receivers(self, choice, name):
        receivers = self.load_list(self.receivers_file)
        if choice == '1':  # Add receiver
            if name not in receivers:
                receivers.append(name)
                self.save_list(self.receivers_file, receivers)
                messagebox.showinfo("Receiver Added", f"Receiver '{name}' added.")
            else:
                messagebox.showinfo("Info", f"Receiver '{name}' already exists.")
        else:  # Remove receiver
            if name in receivers:
                receivers.remove(name)
                self.save_list(self.receivers_file, receivers)
                messagebox.showinfo("Receiver Removed", f"Receiver '{name}' removed.")
            else:
                messagebox.showwarning("Not Found", f"Receiver '{name}' does not exist.")


In [19]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime
import json
import os

# --- Feedback Manager ---

class FeedbackManager:
    def __init__(self):
        self.file_path = "feedbacks.json"

    def load_feedbacks(self):
        if not os.path.exists(self.file_path):
            return []
        try:
            with open(self.file_path, "r") as f:
                return json.load(f)
        except json.JSONDecodeError:
            return []

    def save_feedbacks(self, feedbacks):
        with open(self.file_path, "w") as f:
            json.dump(feedbacks, f, indent=4)

    def add_feedback(self, name, rating, comment, subject, compulsory_answers):
        date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        feedback = {
            "date": date, "name": name, "rating": rating, "comment": comment,
            "subject": subject, "answers": compulsory_answers
        }
        feedbacks = self.load_feedbacks()
        feedbacks.append(feedback)
        self.save_feedbacks(feedbacks)

    def delete_feedback(self, date, name, rating, comment):
        feedbacks = self.load_feedbacks()
        new_feedbacks = [fb for fb in feedbacks if not (
            fb['date'] == date and fb['name'] == name and 
            fb['rating'] == rating and fb['comment'] == comment
        )]
        self.save_feedbacks(new_feedbacks)

# --- Subject Manager ---

class SubjectManager:
    def __init__(self):
        self.file_path = "subjects.json"

    def load_subjects(self):
        if not os.path.exists(self.file_path):
            return []
        try:
            with open(self.file_path, "r") as f:
                return json.load(f)
        except json.JSONDecodeError:
            return []

    def save_subjects(self, subjects):
        with open(self.file_path, "w") as f:
            json.dump(subjects, f, indent=4)

    def add_subject(self, subject):
        subjects = self.load_subjects()
        if subject not in subjects:
            subjects.append(subject)
            self.save_subjects(subjects)

# --- User Manager ---

class UserManager:
    def __init__(self):
        self.file_path = "users.json"
        # Load or initialize users storage
        if not os.path.exists(self.file_path):
            # Pre-populate with default admin and student
            self.users = [
                {"username": "admin", "password": "admin", "role": "Admin", "profile_name": "Administrator"},
                {"username": "student", "password": "student", "role": "Student", "profile_name": "Default Student"}
            ]
            self.save_users()
        else:
            self.users = self.load_users()

    def load_users(self):
        try:
            with open(self.file_path, "r") as f:
                return json.load(f)
        except json.JSONDecodeError:
            return []

    def save_users(self):
        with open(self.file_path, "w") as f:
            json.dump(self.users, f, indent=4)

    def user_exists(self, username):
        return any(u['username'] == username for u in self.users)

    def add_user(self, username, password, role, profile_name):
        if self.user_exists(username):
            return False
        self.users.append({
            "username": username,
            "password": password,
            "role": role,
            "profile_name": profile_name
        })
        self.save_users()
        return True

    def validate_login(self, username, password, role):
        for u in self.users:
            if u['username'] == username and u['password'] == password and u['role'] == role:
                return u  # Return user dict on success
        return None

    def modify_users(self, choice, name):
        if choice == '1':  # Add
            if self.user_exists(name):
                messagebox.showerror("Error", f"User '{name}' already exists.")
            else:
                # Just demo adding with default password, role, profile_name
                self.add_user(name, "defaultpass", "Student", name)
                messagebox.showinfo("Success", f"User '{name}' added.")
        else:  # Remove
            if not self.user_exists(name):
                messagebox.showerror("Error", f"User '{name}' does not exist.")
            else:
                self.users = [u for u in self.users if u['username'] != name]
                self.save_users()
                messagebox.showinfo("Success", f"User '{name}' removed.")

    def modify_receivers(self, choice, name):
        # Same logic for receivers (stored in separate file)
        receivers = self.load_receivers()
        if choice == '1':  # Add
            if name in receivers:
                messagebox.showerror("Error", f"Receiver '{name}' already exists.")
            else:
                receivers.append(name)
                self.save_receivers(receivers)
                messagebox.showinfo("Success", f"Receiver '{name}' added.")
        else:  # Remove
            if name not in receivers:
                messagebox.showerror("Error", f"Receiver '{name}' does not exist.")
            else:
                receivers.remove(name)
                self.save_receivers(receivers)
                messagebox.showinfo("Success", f"Receiver '{name}' removed.")

    def load_receivers(self):
        path = "receivers.json"
        if not os.path.exists(path):
            return []
        try:
            with open(path, "r") as f:
                return json.load(f)
        except json.JSONDecodeError:
            return []

    def save_receivers(self, receivers):
        with open("receivers.json", "w") as f:
            json.dump(receivers, f, indent=4)

# --- Main Application ---

class FeedbackApp:
    def __init__(self, root, user_data):
        self.root = root
        self.root.title("Feedback Management System")
        self.root.geometry("900x650")
        self.root.configure(padx=10, pady=10)

        self.user_data = user_data
        self.role = user_data['role']
        self.profile_name = user_data.get('profile_name', '')
        self.feedback_mgr = FeedbackManager()
        self.subject_mgr = SubjectManager()
        self.user_mgr = UserManager()

        self.style = ttk.Style()
        self.available_themes = self.style.theme_names()
        self.current_theme = tk.StringVar(value=self.style.theme_use())

        # Theme selection frame
        theme_frame = ttk.Frame(root)
        theme_frame.pack(fill='x', pady=5)
        ttk.Label(theme_frame, text=f"Logged in as: {self.profile_name} ({self.role})").pack(side='left', padx=5)
        ttk.Label(theme_frame, text="Select Theme:").pack(side='left', padx=15)

        self.theme_menu = ttk.OptionMenu(
            theme_frame, self.current_theme,
            self.style.theme_use(),
            *self.available_themes,
            command=self.change_theme
        )
        self.theme_menu.pack(side='left')

        # Tabs
        self.tab_control = ttk.Notebook(root)
        self.setup_tabs()
        self.tab_control.pack(expand=1, fill="both", padx=5, pady=5)

        # Bind tab changed to load feedbacks dynamically
        self.tab_control.bind("<<NotebookTabChanged>>", self.on_tab_changed)

        # Logout Button bottom right
        logout_frame = ttk.Frame(root)
        logout_frame.pack(fill='x', side='bottom')
        ttk.Button(logout_frame, text="Logout", command=self.logout).pack(side='right', padx=10, pady=10)

    def logout(self):
        self.root.destroy()
        show_login_dialog()

    def change_theme(self, selected_theme):
        try:
            self.style.theme_use(selected_theme)
        except tk.TclError:
            messagebox.showerror("Theme Error", f"Theme '{selected_theme}' is not available.")

    def setup_tabs(self):
        if self.role == "Student":
            self.create_feedback_tab()
        else:
            self.create_view_tab()
            self.create_user_tab()
            self.create_receiver_tab()
            self.create_subject_tab()
            self.create_subject_view_tab()

    def on_tab_changed(self, event):
        selected_tab = event.widget.tab(event.widget.index("current"))['text']
        if selected_tab == "View Feedbacks" and self.role == "Admin":
            self.load_feedbacks()

    # Feedback submission tab for students
    def create_feedback_tab(self):
        tab = ttk.Frame(self.tab_control)
        self.tab_control.add(tab, text='Submit Feedback')

        form_frame = ttk.Frame(tab, padding=20)
        form_frame.pack(anchor='center')

        ttk.Label(form_frame, text="Name:").grid(row=0, column=0, sticky="e", padx=5, pady=5)
        self.name_entry = ttk.Entry(form_frame, width=40)
        self.name_entry.grid(row=0, column=1, pady=5)
        self.name_entry.insert(0, self.profile_name)  # autofill profile name for student

        ttk.Label(form_frame, text="Subject:").grid(row=1, column=0, sticky="e", padx=5, pady=5)
        self.subject_var = tk.StringVar()
        self.subject_combo = ttk.Combobox(form_frame, textvariable=self.subject_var, state="readonly", width=37)
        self.subject_combo.grid(row=1, column=1, pady=5)
        self.refresh_subjects_dropdown()

        ttk.Label(form_frame, text="Rating (1-5):").grid(row=2, column=0, sticky="e", padx=5, pady=5)
        self.rating_var = tk.StringVar()
        self.rating_combo = ttk.Combobox(form_frame, textvariable=self.rating_var, state="readonly", width=37)
        self.rating_combo['values'] = [1, 2, 3, 4, 5]
        self.rating_combo.grid(row=2, column=1, pady=5)
        self.rating_combo.current(4)

        ttk.Label(form_frame, text="Comments:").grid(row=3, column=0, sticky="ne", padx=5, pady=5)
        self.comment_text = tk.Text(form_frame, width=40, height=5)
        self.comment_text.grid(row=3, column=1, pady=5)

        # Compulsory questions as checkboxes
        ttk.Label(form_frame, text="Compulsory Questions:").grid(row=4, column=0, sticky="ne", padx=5, pady=5)
        self.compulsory_vars = []
        questions = [
            "Did the instructor explain the topic clearly?",
            "Was the course material helpful?",
            "Would you recommend this subject to others?"
        ]
        cb_frame = ttk.Frame(form_frame)
        cb_frame.grid(row=4, column=1, pady=5, sticky="w")
        for q in questions:
            var = tk.IntVar()
            chk = ttk.Checkbutton(cb_frame, text=q, variable=var)
            chk.pack(anchor='w')
            self.compulsory_vars.append((q, var))

        ttk.Button(form_frame, text="Submit Feedback", command=self.submit_feedback).grid(row=5, column=1, pady=15)

    def refresh_subjects_dropdown(self):
        subjects = self.subject_mgr.load_subjects()
        self.subject_combo['values'] = subjects
        if subjects:
            self.subject_combo.current(0)

    def submit_feedback(self):
        name = self.name_entry.get().strip()
        subject = self.subject_var.get()
        rating = self.rating_var.get()
        comment = self.comment_text.get("1.0", "end").strip()
        compulsory_answers = {q: bool(var.get()) for q, var in self.compulsory_vars}

        if not name or not subject or not rating:
            messagebox.showerror("Input Error", "Please fill in all required fields.")
            return

        if not all(compulsory_answers.values()):
            messagebox.showerror("Compulsory", "Please answer all compulsory questions.")
            return

        self.feedback_mgr.add_feedback(name, rating, comment, subject, compulsory_answers)
        messagebox.showinfo("Success", "Feedback submitted successfully!")
        self.comment_text.delete("1.0", "end")
        # reset checkboxes
        for _, var in self.compulsory_vars:
            var.set(0)

    # View Feedbacks for Admin
    def create_view_tab(self):
        tab = ttk.Frame(self.tab_control)
        self.tab_control.add(tab, text="View Feedbacks")
        self.feedback_tab = tab

        columns = ("date", "name", "rating", "comment", "subject")
        self.tree = ttk.Treeview(tab, columns=columns, show="headings", height=15)
        for col in columns:
            self.tree.heading(col, text=col.capitalize())
            self.tree.column(col, anchor="center", width=150)
        self.tree.pack(expand=True, fill='both', padx=10, pady=10)

        btn_frame = ttk.Frame(tab)
        btn_frame.pack(fill='x', pady=5)
        ttk.Button(btn_frame, text="Delete Selected Feedback", command=self.delete_selected_feedback).pack()

    def load_feedbacks(self):
        for item in self.tree.get_children():
            self.tree.delete(item)
        feedbacks = self.feedback_mgr.load_feedbacks()
        for fb in feedbacks:
            self.tree.insert("", "end", values=(
                fb.get("date", ""),
                fb.get("name", ""),
                fb.get("rating", ""),
                fb.get("comment", ""),
                fb.get("subject", "")
            ))

    def delete_selected_feedback(self):
        selected = self.tree.selection()
        if not selected:
            messagebox.showwarning("Select feedback", "Please select feedback to delete.")
            return
        for sel in selected:
            vals = self.tree.item(sel, "values")
            self.feedback_mgr.delete_feedback(vals[0], vals[1], vals[2], vals[3])
        self.load_feedbacks()

    # User Management tab for Admin
    def create_user_tab(self):
        tab = ttk.Frame(self.tab_control)
        self.tab_control.add(tab, text="Manage Users")

        lbl_choice = ttk.Label(tab, text="Choose option: (1) Add, (2) Remove")
        lbl_choice.pack(pady=5)
        self.user_choice_entry = ttk.Entry(tab, width=10)
        self.user_choice_entry.pack(pady=5)

        lbl_name = ttk.Label(tab, text="Enter Username:")
        lbl_name.pack(pady=5)
        self.user_name_entry = ttk.Entry(tab, width=40)
        self.user_name_entry.pack(pady=5)

        ttk.Button(tab, text="Submit", command=self.modify_users).pack(pady=15)

    def modify_users(self):
        choice = self.user_choice_entry.get().strip()
        name = self.user_name_entry.get().strip()
        if choice not in ("1", "2") or not name:
            messagebox.showerror("Input Error", "Please enter valid option and username.")
            return
        self.user_mgr.modify_users(choice, name)
        self.user_choice_entry.delete(0, "end")
        self.user_name_entry.delete(0, "end")

    # Receiver Management tab for Admin
    def create_receiver_tab(self):
        tab = ttk.Frame(self.tab_control)
        self.tab_control.add(tab, text="Manage Receivers")

        lbl_choice = ttk.Label(tab, text="Choose option: (1) Add, (2) Remove")
        lbl_choice.pack(pady=5)
        self.receiver_choice_entry = ttk.Entry(tab, width=10)
        self.receiver_choice_entry.pack(pady=5)

        lbl_name = ttk.Label(tab, text="Enter Receiver Name:")
        lbl_name.pack(pady=5)
        self.receiver_name_entry = ttk.Entry(tab, width=40)
        self.receiver_name_entry.pack(pady=5)

        ttk.Button(tab, text="Submit", command=self.modify_receivers).pack(pady=15)

    def modify_receivers(self):
        choice = self.receiver_choice_entry.get().strip()
        name = self.receiver_name_entry.get().strip()
        if choice not in ("1", "2") or not name:
            messagebox.showerror("Input Error", "Please enter valid option and receiver name.")
            return
        self.user_mgr.modify_receivers(choice, name)
        self.receiver_choice_entry.delete(0, "end")
        self.receiver_name_entry.delete(0, "end")

    # Subject Management tab for Admin
    def create_subject_tab(self):
        tab = ttk.Frame(self.tab_control)
        self.tab_control.add(tab, text="Manage Subjects")

        frame = ttk.Frame(tab, padding=10)
        frame.pack(pady=10)

        ttk.Label(frame, text="Enter Subject Name:").grid(row=0, column=0, padx=5, pady=5)
        self.subject_entry = ttk.Entry(frame, width=40)
        self.subject_entry.grid(row=0, column=1, padx=5, pady=5)

        ttk.Button(frame, text="Add Subject", command=self.add_subject).grid(row=1, column=1, pady=10, sticky="e")

        ttk.Label(frame, text="Current Subjects:").grid(row=2, column=0, sticky="nw", pady=10, padx=5)
        self.subjects_listbox = tk.Listbox(frame, height=8, width=40)
        self.subjects_listbox.grid(row=2, column=1, pady=10, padx=5, sticky="w")

        self.load_subjects_to_listbox()

    def add_subject(self):
        subject = self.subject_entry.get().strip()
        if not subject:
            messagebox.showerror("Input Error", "Please enter a subject name.")
            return
        self.subject_mgr.add_subject(subject)
        self.subject_entry.delete(0, "end")
        self.load_subjects_to_listbox()
        self.refresh_subjects_dropdown()

    def load_subjects_to_listbox(self):
        self.subjects_listbox.delete(0, "end")
        subjects = self.subject_mgr.load_subjects()
        for s in subjects:
            self.subjects_listbox.insert("end", s)

    # For completeness, create a separate tab for subject viewing (optional)
    def create_subject_view_tab(self):
        pass  # Already showing subjects in Manage Subjects tab

# --- Login/Signup Dialog ---

def show_login_dialog():
    def login():
        username = username_entry.get().strip()
        password = password_entry.get().strip()
        role = role_var.get()

        if not username or not password:
            messagebox.showerror("Error", "Please enter username and password.")
            return

        user = user_mgr.validate_login(username, password, role)
        if user:
            login_window.destroy()
            root = tk.Tk()
            FeedbackApp(root, user)
            root.mainloop()
        else:
            messagebox.showerror("Error", "Invalid credentials or role.")

    def signup():
        def perform_signup():
            su_username = su_username_entry.get().strip()
            su_password = su_password_entry.get().strip()
            su_profile_name = su_profile_entry.get().strip()
            su_role = su_role_var.get()
            if not (su_username and su_password and su_profile_name):
                messagebox.showerror("Error", "All fields are required.")
                return
            if user_mgr.user_exists(su_username):
                messagebox.showerror("Error", "Username already exists.")
                return
            user_mgr.add_user(su_username, su_password, su_role, su_profile_name)
            messagebox.showinfo("Success", "User created successfully.")
            signup_window.destroy()

        signup_window = tk.Toplevel(login_window)
        signup_window.title("Sign Up")

        ttk.Label(signup_window, text="Username:").grid(row=0, column=0, padx=5, pady=5)
        su_username_entry = ttk.Entry(signup_window)
        su_username_entry.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(signup_window, text="Password:").grid(row=1, column=0, padx=5, pady=5)
        su_password_entry = ttk.Entry(signup_window, show="*")
        su_password_entry.grid(row=1, column=1, padx=5, pady=5)

        ttk.Label(signup_window, text="Profile Name:").grid(row=2, column=0, padx=5, pady=5)
        su_profile_entry = ttk.Entry(signup_window)
        su_profile_entry.grid(row=2, column=1, padx=5, pady=5)

        ttk.Label(signup_window, text="Role:").grid(row=3, column=0, padx=5, pady=5)
        su_role_var = tk.StringVar(value="Student")
        ttk.Radiobutton(signup_window, text="Admin", variable=su_role_var, value="Admin").grid(row=3, column=1, sticky="w")
        ttk.Radiobutton(signup_window, text="Student", variable=su_role_var, value="Student").grid(row=4, column=1, sticky="w")

        ttk.Button(signup_window, text="Sign Up", command=perform_signup).grid(row=5, column=1, pady=10, sticky="e")

    login_window = tk.Tk()
    login_window.title("Login")
    login_window.geometry("350x250")
    login_window.resizable(False, False)

    user_mgr = UserManager()  # Create user manager instance

    ttk.Label(login_window, text="Username:").pack(pady=5)
    username_entry = ttk.Entry(login_window)
    username_entry.pack(pady=5)

    ttk.Label(login_window, text="Password:").pack(pady=5)
    password_entry = ttk.Entry(login_window, show="*")
    password_entry.pack(pady=5)

    ttk.Label(login_window, text="Role:").pack(pady=5)
    role_var = tk.StringVar(value="Student")
    ttk.Radiobutton(login_window, text="Admin", variable=role_var, value="Admin").pack()
    ttk.Radiobutton(login_window, text="Student", variable=role_var, value="Student").pack()

    ttk.Button(login_window, text="Login", command=login).pack(pady=10)
    ttk.Button(login_window, text="Sign Up", command=signup).pack()

    login_window.mainloop()

if __name__ == "__main__":
    show_login_dialog()



In [12]:
import json
import os
from datetime import datetime

class FeedbackManager:
    def __init__(self, file_path="feedbacks.json"):
        self.file_path = file_path

    def load_feedbacks(self):
        """Load feedbacks from the JSON file."""
        if not os.path.exists(self.file_path):
            return []
        with open(self.file_path, "r") as f:
            try:
                return json.load(f)
            except json.JSONDecodeError:
                return []

    def save_feedbacks(self, feedbacks):
        """Save feedbacks to the JSON file."""
        with open(self.file_path, "w") as f:
            json.dump(feedbacks, f, indent=4)

    def add_feedback(self, feedback):
        """Add a feedback entry to the JSON file."""
        feedbacks = self.load_feedbacks()
        feedbacks.append(feedback)
        self.save_feedbacks(feedbacks)

    def delete_feedback(self, date, name, rating, comment):
        """Delete a feedback entry by matching date, name, rating, and comment."""
        feedbacks = self.load_feedbacks()
        new_feedbacks = [fb for fb in feedbacks if not (
            fb['date'] == date and fb['name'] == name and 
            fb['rating'] == rating and fb['comment'] == comment
        )]
        self.save_feedbacks(new_feedbacks)

    def get_feedback_summary(self):
        """Calculate and return the average rating and total count of feedbacks."""
        feedbacks = self.load_feedbacks()
        if feedbacks:
            average_rating = sum(fb['rating'] for fb in feedbacks) / len(feedbacks)
            return average_rating, len(feedbacks)
        return 0, 0



In [10]:
import json

def check_login(username, password, role):
    try:
        with open('users.json', 'r') as f:
            users = json.load(f)
    except FileNotFoundError:
        return False

    for user in users:
        if user['username'] == username and user['password'] == password and user['role'] == role:
            return True
    return False
